# `swift_too` module

## Swift_QueryJob example - Querying and fetching the results of an existing job

### API version = 1.0

#### Author: Jamie A. Kennea (Penn State)

### Introduction
If you wish to retrieve the results of a previously job, this is the task of the QueryJob class. All you need for this class is your username, shared secret and the job number which is reported back when the job is submitted either with `submit()` or `queue()` methods. The jobnumber can be found in the property `jobnumber`.

In [1]:
from swifttools.swift_too import QueryJob, Swift_VisQuery
from time import sleep

I'll define our username and shared secret next because we'll use them twice in this notebook

In [2]:
username = 'myuser'
shared_secret = 'mysharedsecret'

### Generating a job to query

First let's submit a simple job so we can get an example jobnumber. In this case I'm just going to do something that isn't too strenuous on the system, let's see if Sgr A* is currently visible, or when it is next visible.

In [3]:
vq = Swift_VisQuery()
vq.username = username
vq.shared_secret = shared_secret
vq.ra,vq.dec = 266.4168, -29.0078
vq.length = 1
if vq.queue():
    print(f"Queued job successfully! Jobnumber = {vq.status.jobnumber}")
else:
    print(f"Error: {vq.status.errors}")

Queued job successfully! Jobnumber = 3052


Note that the job has been submitted, but because we used the `queue()` method, it may not have completed yet. If you wait a bit, it will be, but the next step can be run straight away to demonstrate what happens if processing isn't complete.

### Querying job status

OK let's see how our job is doing, because the purpose of the QueryJob class is to allow us to fetch a job later, or maybe ASAP. Maybe you don't want to hang around waiting for a bunch of jobs to complete, submit some, have some coffee, come back and fetch, or just want to query the result of an existing job.

In [4]:
qj = QueryJob()
qj.username = username
qj.shared_secret = shared_secret
qj.jobnumber = vq.status.jobnumber
if qj.submit():
    print(f"Current status of job #{qj.jobnumber}: {qj.status}")
else:
    print(f"Request {qj.status}. Errors: {qj.errors}")

Current status of job #3052: Queued


The `submit` should return `True` if the job status has been fetched. It'll return `False` only in the case where any of the given parameters are incorrect, for example a wrong `username` and `shared_secret` combo, or a non-existant `jobnumber`. 

The current `status` can have several return values. Firstly if the job is still *Queued* (waiting to be processed) or *Processing*, the status will indicate this. Also the status can be *Rejected*, in the case where you have given bad parameters, or the job no longer exists.


Note that for the class `QueryJob`, as it does not take many parameters, you can also pass parameters as arguments. If you do this, then the request will be submitted on invokation, so there's no need use the `submit()` method. 

The next code entry demonstrates this usage, and also acts to make sure that the job is marked as *Accepted*. It also demonstrates the boolean usage of the class, which can be used to determine if a job is *Accepted*, and is false if *Rejected*, *Queued* or *Processing*. 

In [5]:
qj = False
while(not qj):
    sleep(1)
    qj = QueryJob(username,shared_secret,vq.status.jobnumber)

print(f"Current status of job #{qj.jobnumber}: {qj.status}")

Current status of job #3052: Accepted


note that if you were using Python 3.8 or later, you could simplify the above status using the := operator, example given below.

In [6]:
while(not (qj := QueryJob(username,shared_secret,vq.status.jobnumber))):
    sleep(1)

print(f"Current status of job #{qj.jobnumber}: {qj.status}")

Current status of job #3052: Accepted


OK now the above should report "Current status of job #XXX: Accepted", which indicates a successful `QueryJob` fetching of a completed job. 

### Examining the results of a QueryJob

Let's see what our QueryJob class data now looks like.

In [7]:
qj.api_data

{'username': 'myuser',
 'status': 'Accepted',
 'jobnumber': 3052,
 'errors': [],
 'warnings': [],
 'timestamp': '2021-03-14 13:37:16',
 'began': '2021-03-14 13:37:17',
 'completed': '2021-03-14 13:37:17',
 'fetchresult': True,
 'result': {'api_name': 'Swift_VisQuery',
  'api_version': '1.0',
  'api_data': {'username': 'myuser',
   'ra': 266.4168,
   'dec': -29.0078,
   'begin': '2021-03-14 13:37:14.526523',
   'length': 1,
   'hires': False,
   'windows': [{'api_name': 'Swift_VisWindow',
     'api_version': '1.0',
     'api_data': {'begin': '2021-03-14 13:37:00',
      'end': '2021-03-15 13:37:00'}}]}}}

You can see the data inside the class here as dict. The the mode useful value here is `result`, which contains the result of the job. In the Python api, this will be a `Swift_VisQuery` object.

In [8]:
qj.result

<Swift_VisQuery: ['username=myuser', 'ra=266.4168', 'dec=-29.0078', 'begin=2021-03-14 13:37:14.526523', 'length=1', 'hires=False']>

Other parameters that might be of interest are the `timestamp`, `began` and `completed` values, which allow you to find out when the job was submitted, when it started processing and when it finished. For example:

In [9]:
print(f"Job was submitted at {qj.timestamp}, was queued for {(qj.began - qj.timestamp).seconds}s and took {(qj.completed - qj.began).seconds}s to process.")

Job was submitted at 2021-03-14 13:37:16, was queued for 1s and took 0s to process.


As usual, times reported back are UT. That is it! If all is correct, then the following show the upcoming or current *Swift* visibility window for Sgr A*!

In [10]:
_ = [print(win) for win in qj.result]

2021-03-14 13:37:00 - 2021-03-15 13:37:00


That first date is going to be the first visibility date, which if the target is visible now should be essentially the same as `timestamp` in `QueryJob`, but rounded down to the nearest minute, or sometime in the future if the target was constrained when the job was submitted. 

In [11]:
(qj.result[0][0] - qj.timestamp)

datetime.timedelta(days=-1, seconds=86384)